In [1]:
# Import the IMDb class from the imdb package
from imdb import IMDb
import requests
import time

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # Import the By class
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import import_ipynb
from user import User
import db


from faker import Faker
import random

from config import TMDB_API_KEY
from imdb import Cinemagoer
from collections import deque

from datetime import datetime

from filemanager import FileManager
from q import Queue
from web_spider import WebSpider

import threading

print(TMDB_API_KEY)  # Just to verify it's imported correctly; remove in production

importing Jupyter notebook from db.ipynb
 cf_model.py	      q.py
 completedstack.txt   qtest.txt
 config.py	      README.md
 dataset.py	      requirements.txt
 db.ipynb	     'Spider-4_doing_The Super Mario Bros. Movie.txt'
 done.txt	      Spider-4_doing_Titanic.txt
 done.txt.lock	      todo.txt
 fdstests.ipynb       todo.txt.lock
 filemanager.py       train.ipynb
 main.ipynb	      two
 menv		      user.py
 model_weights.pth    wandb
 movusers.db	      web_spider.py
 one		      your_database.db
 __pycache__
e47716bb6a8a0524a757f0b9320f0b29


In [2]:
#common practise to have users rate at least 20-30 reviews each....
TOTAL_MOVIES = 30
TOTAL_USERS = 2400

In [3]:
def fetch_movie_names(api_key=TMDB_API_KEY, total_movies=10):
    
    base_url = "https://api.themoviedb.org/3/discover/movie"
    movie_count = 0
    page = 1
    while movie_count < total_movies:
        params = {
            "api_key": api_key,
            "language": "en-US",
            "sort_by": "revenue.desc",
            "page": page
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            for movie in data['results']:

                movie_count += 1
                yield movie['title']
                if movie_count >= total_movies:
                    break
        else:
            print(f"Failed to fetch movies: {response.status_code}")
            break
        page += 1
        time.sleep(1)  # Respect the rate limit
        
    # movie_names[:total_movies]



In [4]:
def scrape_imdb_movie_reviews(movie_id):

    # Specify the path to chromedriver
    service = Service(executable_path='/home/james/.wdm/drivers/chromedriver/linux64/121.0.6167.85/chromedriver-linux64/chromedriver')



    # Create ChromeOptions object
    chrome_options = webdriver.ChromeOptions()
    
    # Disable popups
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')  # Disable GPU accelerati


    
    
    # Initialize the WebDriver with the Service object
    driver = webdriver.Chrome(service=service,options = chrome_options)

    # url = 'https://www.imdb.com/title/tt'+movie_id+'/reviews'
    # print(url)
    # Open a webpage
    driver.get('https://www.imdb.com/title/tt'+movie_id+'/reviews')

    
    # Wait for the page to load (optional, could use more sophisticated waits)
    time.sleep(5)

    # Wait for the cookies consent button to be clickable, and then click it
    wait = WebDriverWait(driver, 30)
    cookies_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Accept"]')))
    cookies_button.click()
        
    # Example: Find an element and retrieve text (update the selector as needed)
    
    # THE PAGE MIGHT HAVE A load more of the reviews getting the webdriver to test press the button.

    try:
        max_count = 100
        count = 0
        
        while count < max_count:  # Keep clicking the "Load More" button until it's no longer found
            time.sleep(1)

            wait = WebDriverWait(driver, 30)
            load_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Load More"]')))
            load_button.click()
            
            count += 1

            elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'lister-item-content')]")
            # print(elements)
        #TODO fix line below
        # reviews = [rv for rv in get_imdb_reviews(elements,movie['id'])]


        #NOTE, webspider not handling submodularisation
        #--------------------------------------------------------------------------------------------------------
        review = {}
        
        for element in elements:
            # print(element)
            #return the individual reviews with their respective ratings
            edivs = element.find_elements(By.TAG_NAME, 'div')
            title_div = element.find_elements(By.TAG_NAME, 'a')

            review = {'ratings':None,'check':''}
            review['title'] = title_div[0].text
            review['movie_id'] = movie_id

            # Step 3: Retrieve and print the text for each child element
            details = {0:'ratings',1:'author',2:'content',3:'check'}
            count = 0

            for div in edivs:
                # print(div.get_attribute('outerHTML'))
                if count in [0,1,2,3]:
                    #possible that there is no rating so check forms.
                    if count == 0:
                        #"10/10"
                        try:
    
                            int(div.text.split('/')[0])
                            review[details[count]] = div.text
                        
                        except:
                            
                            print('review has no rating, skipping')
                            break
                            
                    elif count == 1:
                        data = div.text.split()
                        
                        # print(data)
                        if len(data) >= 1:
                            review['author'] = data[0]
                        if len(data) >= 2:
                            review['date'] = ' '.join(data[1:])
                    else:
    
                        review[details[count]] = div.text
    
                    count +=1
    
    
    
            if review['check'] == '':
                #no actual review in these cases.
                review['content'] = None
                
            del review['check']
            # print(review)
            if review['ratings'] != None:
                yield review


    except Exception as e:
        print("All content loaded or button not found.")
    
    
    finally:

    # Close the browser
        driver.quit()

In [5]:

# mix of imdb and tmdb
def search_movie_by_name(movie_name, api_key=TMDB_API_KEY):
    """Searches for a movie by name on TMDb, prints the first result, and fetches reviews including ratings."""
    # Search for the movie

    movie = {'title':None,'release_date':None,'genre':'','director':'','actors':[],'ratings':0,'votes':0,'overview':None,'id':None, 'reviews' : []}
    ia = Cinemagoer()
    # ia = IMDb()
    movies = ia.search_movie(movie_name)
    if movies:
    # Get the first movie from the list
        
        mov = movies[0]
        movkeys = mov.keys()
        # print(movkeys)
        # Retrieve detailed information about the movie

        movie['title'] = movie_name
        ia.update(mov)
        if 'genre' in movkeys:
            movie['genre'] = ', '.join(mov['genres'])
        if 'year' in movkeys:
            movie['release_date'] = datetime(mov['year'],1,1)
        if 'directors' in movkeys:
            movie['director'] = ', '.join([director['name'] for director in mov['directors']])
        #temporary, even if empty string.
        movie['director_id'] = id(movie['director'])
            
        if 'actors' in movkeys:
            movie['actors'] = [cast['name'] for cast in mov['cast']][:5]
        if 'plot' in movkeys:        
            movie['story'] = mov['plot'][0]
        
        if 'ratings' in movkeys and 'votes' in movkeys:
            
            movie['ratings'] = mov.get('rating')
            movie['votes'] = mov.get('votes')

        movie['id'] = mov['imdbID']
        return movie
    #fill in the details

    else:
        print("No movies found.")

# search_movie_by_name('Avengers: Endgame')

In [6]:
def scrape_function(movie_name):

    movie = search_movie_by_name(movie_name)
    movie['reviews'] = [rv for rv in scrape_imdb_movie_reviews(movie['id'])]
    return movie

In [7]:
!cat todo.txt

In [8]:
movies_q = Queue()

def run_spider(spider):
    while not FileManager.is_empty('todo.txt'):
        task = spider.get_from_file('todo.txt')
        if task:
            #returns review
            movie_info = spider.scrape(task)  # Assuming this method now handles the scraping logic
            movies_q.enqueue(movie_info)
            spider.push_to_file(task, 'done.txt')
        else:
            break

In [9]:
!cat done.txt

Iron Man 3
Barbie
Spider-Man: Far From Home
Furious 7
Incredibles 2
Jurassic World: Fallen Kingdom
Avatar: The Way of Water
Avengers: Infinity War
Minions
Beauty and the Beast
Black Panther
Titanic
Avatar
Aquaman
Star Wars: The Last Jedi
Spider-Man: No Way Home
The Fate of the Furious
Star Wars: The Force Awakens
Frozen II
Skyfall
Captain America: Civil War
Jurassic World
Harry Potter and the Deathly Hallows: Part 2
Top Gun: Maverick
Frozen
Avengers: Age of Ultron
The Lion King
The Avengers
Avengers: Endgame


In [10]:
def generate_users(cc,count):

    for i in range(0,count):
        yield User(Faker(random.choice(cc)).name())


# for us in generate_users(["ar_AA"],1):
#     print(us.name())

In [11]:
# TODO content filter
# 
# genre and country,.
# reverse the matching, not most suited, not least suited, 

In [12]:

def main():
        
    db.drop_tables()
    db.create_db()
    
    #create a set of movie_names
    movie_names = list({mn for mn in fetch_movie_names(total_movies = TOTAL_MOVIES)})
    print(movie_names)
    fm = FileManager()
    fm.write_file('todo.txt')
    fm.write_file('done.txt')

    for movie_name in movie_names:

        fm.enqueue_to_file(movie_name,'todo.txt')


    country_codes = [
        "ar_AA", "az_AZ", "bg_BG", "bs_BA", "cs_CZ", "de_DE", "dk_DK", "el_GR",
        "en_AU", "en_CA", "en_GB", "en_IN", "en_NZ", "en_US", "es_ES", "es_MX",
        "et_EE", "fa_IR", "fi_FI", "fr_FR", "hi_IN", "hr_HR", "hu_HU", "hy_AM",
        "id_ID", "it_IT", "ja_JP", "ka_GE", "ko_KR", "lt_LT", "lv_LV", "ne_NP", 
        "nl_NL", "no_NO", "pl_PL", "pt_BR", "pt_PT", "ro_RO", "ru_RU", "sk_SK",
        "sl_SI", "sv_SE", "tr_TR", "uk_UA", "zh_CN", "zh_TW"
    ]
    
    users = [user for user in generate_users(country_codes,TOTAL_USERS)]
    
    #real work, get the movies themselves and reviews.
    spiders = [WebSpider(file_manager=FileManager, scrape_function=scrape_function, name=f"Spider-{i}") for i in range(5)]
    
    threads = [threading.Thread(target=run_spider, args=(spider,)) for spider in spiders]
    
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()

    # while not movies_q.is_empty():
        
    #     print('checking ds')
    
    #     print(type(movies_q))
    #     #list
    #     print(type(movies_q.items))
    #     print(len(movies_q.items))
    #     mov = movies_q.dequeue()
    #     # print('mov is :',mov)
    #     # print('type is :',type(mov))
    #     # print('movrs are:',mov['reviews'])
    #     # print(len(mov['reviews']))
    
    #     print('putting into q', Queue(mov['reviews']))
    

    import copy
    
    copy_mq = copy.deepcopy(movies_q)
    user_q = Queue(users)
    leave_q = Queue([])
    
    
    while not movies_q.is_empty():
        mov = movies_q.dequeue()
        
        mov_rq = Queue(mov['reviews'])
        
        #shuffle users before handing out
        random.shuffle(user_q.items)
        # print(len(user_q.items))
        while not user_q.is_empty():
            # if there is anything to hand out.
            user = user_q.dequeue()
            # print(user)
            #we need to hand out to them.....
            if not mov_rq.is_empty():
                review = mov_rq.dequeue()
                review['user_id'] = user.user_id
                user.add_review(review)
                # print(user.name,'has new review',review)
            leave_q.enqueue(user)
            # print(len(user_q))
            # print(len(leave_q))

        #done for that subqueue
        user_q = leave_q
        leave_q = Queue([])

    # print(user_q.items[:10])
    # print(copy_mq.is_empty())

    #now do the DB

    while not user_q.is_empty():
        user = user_q.dequeue()
        db.add_user(user)
        # print(user.reviews)
        for rv in user.reviews:
            db.add_movie_review(rv)

    
    while not copy_mq.is_empty():

        movie = copy_mq.dequeue()
        
        db.add_movie(movie)






    

In [13]:
main()

SQLite database and table created locally.
['Minions', 'Captain America: Civil War', 'Spider-Man: Far From Home', 'Skyfall', 'Harry Potter and the Deathly Hallows: Part 2', 'Top Gun: Maverick', 'Titanic', 'Spider-Man: No Way Home', 'Black Panther', 'Avengers: Infinity War', 'Avatar', 'Star Wars: The Last Jedi', 'The Lion King', 'Frozen', 'Beauty and the Beast', 'Avengers: Endgame', 'Incredibles 2', 'Aquaman', 'Frozen II', 'The Avengers', 'Avatar: The Way of Water', 'Avengers: Age of Ultron', 'Star Wars: The Force Awakens', 'Jurassic World: Fallen Kingdom', 'The Fate of the Furious', 'Jurassic World', 'Iron Man 3', 'Furious 7', 'Barbie', 'The Super Mario Bros. Movie']
File 'todo.txt' created successfully
File 'done.txt' created successfully
Item 'Minions' successfully enqueued to todo.txt
Item 'Captain America: Civil War' successfully enqueued to todo.txt
Item 'Spider-Man: Far From Home' successfully enqueued to todo.txt
Item 'Skyfall' successfully enqueued to todo.txt
Item 'Harry Potte

2024-03-04 21:32:22,989 CRITICAL [imdbpy] /home/james/Desktop/torchex/movie-users/menv/lib/python3.10/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=Aquaman&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/home/james/Desktop/torchex/movie-users/menv/lib/python3.10/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391

review has no rating, skipping


Exception in thread Thread-5 (run_spider):
Traceback (most recent call last):
  File "/home/james/Desktop/torchex/movie-users/menv/lib/python3.10/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "/usr/lib/python3.10/urllib/request.py", line 1352, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason 

review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review has no rating, skipping
review h

In [14]:
print(db.display_size())

Users: 2400

Movies: 29

Reviews: 23570
None


In [15]:
# print(db.display_data())